<a href="https://colab.research.google.com/github/varajan2007/public_vani/blob/master/lstm_glove_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


Selection tensor flow 2.o updated version


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import math
%matplotlib inline

In [0]:
!git clone https://github.com/varajan2007/public_vani.git

fatal: destination path 'public_vani' already exists and is not an empty directory.


In [0]:
df = pd.read_csv("TweetsAirlines.csv")

In [0]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570000000000000000,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24-02-15 11:35,NaN,Eastern Time (US & Canada)
1,570000000000000000,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24-02-15 11:15,NaN,Pacific Time (US & Canada)
2,570000000000000000,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,24-02-15 11:15,Lets Play,Central Time (US & Canada)
3,570000000000000000,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,24-02-15 11:15,NaN,Pacific Time (US & Canada)
4,570000000000000000,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,24-02-15 11:14,NaN,Pacific Time (US & Canada)


In [0]:
df.shape
df.isnull().sum()
df[['text', 'airline_sentiment']].head()
df['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [0]:
df['airline_sentiment'] = df['airline_sentiment'].replace('neutral', 1)
df['airline_sentiment'] = df['airline_sentiment'].replace('negative', 0)
df['airline_sentiment'] = df['airline_sentiment'].replace('positive', 2)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [0]:
# Convert training data into tensors to feed into neural net
# create tokenizer  Split into text and label
X = df['text'] # data
y = df['airline_sentiment'] # labels
t = Tokenizer()
t.fit_on_texts(X)

In [0]:
vocab_size = len(t.word_index) + 1
sequences = t.texts_to_sequences(X)
# Find longest tweet in sequences
def max_tweet():
    for i in range(1, len(sequences)):
        max_length = len(sequences[0])
        if len(sequences[i]) > max_length:
            max_length = len(sequences[i])
    return max_length
tweet_num = max_tweet()
tweet_num
# each review has a different number of words, so pad sequence with 0's
# https://realpython.com/python-keras-text-classification/
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = tweet_num
padded_X = pad_sequences(sequences, padding='post', maxlen=maxlen)

In [0]:

labels = to_categorical(np.asarray(y))
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_X, labels,
                                                    test_size = 0.2, random_state = 0)

In [0]:
# Size of train and test datasets
print('X_train size:', X_train.shape)
print('y_train size:', y_train.shape)
print('X_test size:', X_test.shape)
print('y_test size:', y_test.shape)

X_train size: (11712, 30)
y_train size: (11712, 3)
X_test size: (2928, 30)
y_test size: (2928, 3)


In [0]:
# load the whole embedding into memory
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
# 100 dimensional version (embedding dimension)
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 3672 word vectors.


In [0]:
 #Because we're using GloVE, need to create a word embedding/word context matrix
# Create a matrix of weights for words in the training set
# One embedding for each word in the training set

# Get all unique words in our training set: Tokenizer index
# Find the corresponding weight vector in GloVe embedding

# Define size of embedding matrix: number of unique words x embedding dim (100)
embedding_matrix = np.zeros((vocab_size, 100))

# fill in matrix
for word, i in t.word_index.items():  # dictionary
    embedding_vector = embeddings_index.get(word) # gets embedded vector of word from GloVe
    if embedding_vector is not None:
        # add to matrix
        embedding_matrix[i] = embedding_vector # each row of matrix

In [0]:
# Create embedding layer using embedding matrix
from tensorflow.keras.layers import Embedding

# input is vocab_size, output is 100
# weights from embedding matrix, set trainable = False
embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix],
                           input_length = tweet_num, trainable=False)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import BatchNormalization

In [0]:
lstm_mod1 = Sequential()
lstm_mod1.add(embedding_layer)
lstm_mod1.add(LSTM(256, 
               dropout = 0.2, 
               recurrent_dropout = 0.5))
lstm_mod1.add(Dense(3, activation='softmax'))
lstm_mod1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
lstm_mod1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 100)           1576900   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dense (Dense)                (None, 3)                 771       
Total params: 1,943,239
Trainable params: 366,339
Non-trainable params: 1,576,900
_________________________________________________________________


In [0]:
hist_1 = lstm_mod1.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs=100, batch_size=256)

Train on 9369 samples, validate on 2343 samples
Epoch 1/100
9369/9369 [==============================] - 22s 2ms/sample - loss: 0.8681 - acc: 0.6172 - val_loss: 0.8385 - val_acc: 0.6270
Epoch 2/100
9369/9369 [==============================] - 19s 2ms/sample - loss: 0.8335 - acc: 0.6483 - val_loss: 0.7986 - val_acc: 0.6620
Epoch 3/100
9369/9369 [==============================] - 19s 2ms/sample - loss: 0.7945 - acc: 0.6711 - val_loss: 0.7444 - val_acc: 0.7059
Epoch 4/100
9369/9369 [==============================] - 19s 2ms/sample - loss: 0.7730 - acc: 0.6827 - val_loss: 0.7232 - val_acc: 0.7106
Epoch 5/100
9369/9369 [==============================] - 19s 2ms/sample - loss: 0.7597 - acc: 0.6873 - val_loss: 0.7155 - val_acc: 0.7055
Epoch 6/100
9369/9369 [==============================] - 19s 2ms/sample - loss: 0.7482 - acc: 0.6954 - val_loss: 0.7092 - val_acc: 0.7055
Epoch 7/100
9369/9369 [==============================] - 19s 2ms/sample - loss: 0.7433 - acc: 0.6947 - val_loss: 0.6899 - va